In [1]:
using PyPlot
using Distributed
# addprocs();

In [2]:

@everywhere using Revise
using CSV
using JSON
using HDF5

@everywhere using RealNeuralNetworks
@everywhere using RealNeuralNetworks.Neurons
@everywhere using RealNeuralNetworks.SWCs
@everywhere using RealNeuralNetworks.NBLASTs
@everywhere using RealNeuralNetworks.Neurons.Segments.Synapses
@everywhere using RealNeuralNetworks.Utils.RangeIndexingArrays

@everywhere using ProgressMeter

@everywhere DATASET_DIR = expanduser("~/seungmount/research/Jingpeng/14_zfish/01_data/20190415")

┌ Info: Recompiling stale cache file /usr/people/jingpeng/.julia/compiled/v1.3/RealNeuralNetworks/iv4AJ.ji for RealNeuralNetworks [4491297b-8966-5840-8cb9-b189d60f3398]
└ @ Base loading.jl:1240
┌ Warning: Package MetaGraphs does not have Random in its dependencies:
│ - If you have MetaGraphs checked out for development and have
│   added Random as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with MetaGraphs
└ Loading Random into MetaGraphs from project dependency, future warnings for MetaGraphs are suppressed.
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for RealNeuralNetworks [4491297b-8966-5840-8cb9-b189d60f3398]
│   exception = ErrorException("Required dependency MetaGraphs [626554b9-1ddb-594c-aa3c-2596fe9399a5] failed to load from a cache file.")
└ @ Base loading.jl:1025
┌ Info: Recompiling stale cache file /usr/people/jingpeng/.julia/compiled

In [3]:
# df = CSV.read("$(DATASET_DIR)/../groundtruth/ManualSorting_forEvaluation.csv", header = 2, skipto=3)

# manualClustering = Dict{Symbol, Vector{Int}}()

# for neuronTypeName in names(df)
#     col = df[neuronTypeName]
#     # drop missing values
#     k = findfirst(ismissing, col)
#     if k != nothing
#         col = col[1: k-1 ]
#     end
# #     @show col
#     if isa(col, Vector{Union{Missing, String}})
#         col = map(Meta.parse, col)
#     end
#     manualClustering[neuronTypeName] = col
# end 

# # eliminate the neurons which are not valid and was not skeletonized
# for (typeName, neuronIdList) in manualClustering
#     notSkeletonizedNeuronIdIndex = Vector{Int}()
#     for (index, neuronId) in neuronIdList |> enumerate
#         if !isfile(joinpath(DATASET_DIR, "swc.bin/$(neuronId).swc.bin"))
#             push!(notSkeletonizedNeuronIdIndex, index)
#         end
#     end
#     deleteat!(neuronIdList, notSkeletonizedNeuronIdIndex)
# end


using JSON
manualClustering = JSON.parsefile("$(DATASET_DIR)/../groundtruth/manual_sorting.json", 
                                                        dicttype=Dict{Symbol, Vector{Int}})

neuronIdList = Int[]
for (groupName,idList) in manualClustering
    append!(neuronIdList, idList)
end
println("got $(length(neuronIdList)) neurons.")

# detect duplicates
neuronIdSet = Set{Int}(neuronIdList)
for (i,neuronId1) in enumerate(neuronIdList)
    for neuronId2 in neuronIdList[i+1:end]
        if neuronId1 == neuronId2
            println("detected duplicated neuron: ", neuronId1)
        end
    end 
end

manualClustering

got 209 neurons.


Dict{Symbol,Array{Int64,1}} with 10 entries:
  :AbdC                        => [77154, 77646, 77682, 77628, 77295, 77652, 77…
  :LateralDorsal               => [76629, 76667, 80185, 76691, 76692, 77357, 77…
  :DivingAxon                  => [76784, 76783, 77394, 77326, 77772, 77766, 77…
  :ABDiR                       => [77631, 77150, 77618, 77886, 78547, 77158, 78…
  :unknown                     => [82188, 82178, 82171, 82179, 82181, 82175, 82…
  :Bushy                       => [76618, 76625, 76627, 77132, 77162, 77163, 77…
  :ABDiC                       => [77148, 77625, 77641, 77692, 77144, 77643, 77…
  :TangentialVestibularNeurons => [78046, 78049, 78048, 78047, 78050, 78051, 78…
  :IO                          => [77483, 77484, 77485, 77486, 77487, 77488, 77…
  :AbdR                        => [77648, 77710, 77300, 77705, 77305, 77301, 77…

In [4]:
testGroups = Symbol[:Bushy, :LateralDorsal, :IO, :DivingAxon, :unknown]

neuronIdList = Int[]
groundTruthPartition = Int[]
for (groupId, groupName) in enumerate(testGroups)
    idList = manualClustering[groupName]
    append!(neuronIdList, idList)
    append!(groundTruthPartition, ones(Int, length(idList)).*groupId)
end

println("neuron ID list: ", neuronIdList)
println("ground truth partition: ", groundTruthPartition)

println("$(length(neuronIdList)) test neurons  in $(length(testGroups)) groups")

# h5open("$(DATASET_DIR)/evaluate.h5", "cw") do f
#     f["neuronIdList"] = neuronIdList
#     f["groundTruthPartition"] = groundTruthPartition
# end

# read neurons

neuronList = pmap(x->Neurons.load("$(DATASET_DIR)/bin/$(x).bin"), neuronIdList);

neuron ID list: [76618, 76625, 76627, 77132, 77162, 77163, 77329, 77374, 77434, 77447, 77460, 77467, 77797, 77805, 77848, 78357, 78358, 79054, 79055, 79059, 79062, 79074, 79077, 79085, 78544, 76629, 76667, 80185, 76691, 76692, 77357, 77389, 77390, 77621, 77636, 77651, 77656, 77689, 77806, 77816, 78601, 78650, 77667, 77684, 77483, 77484, 77485, 77486, 77487, 77488, 77489, 77490, 77492, 77493, 77494, 77495, 77496, 77497, 76353, 77499, 77500, 77501, 77502, 77503, 77504, 77505, 77506, 77507, 77508, 77509, 77510, 77511, 77512, 77513, 77514, 77515, 77517, 77521, 77523, 77524, 77525, 77526, 77527, 77528, 77529, 77530, 77531, 77532, 77533, 77534, 77535, 77536, 77537, 77538, 77539, 78420, 76350, 76271, 76487, 76355, 76351, 76273, 76272, 76488, 76261, 76266, 78395, 78401, 76784, 76783, 77394, 77326, 77772, 77766, 77756, 77753, 77767, 77775, 77780, 80883, 80247, 80669, 80698, 80762, 80748, 80696, 80761, 80749, 81070, 81044, 80991, 80967, 81008, 81045, 80986, 81023, 78647, 78619, 81141, 76664, 821

## NBLAST similarity computation

In [5]:
using HDF5
println("remove the dataset first, otherwise we can not write!")

h5open("$(DATASET_DIR)/evaluate.h5", "r+") do f
    o_delete(f, "flyTable/rawSimilarityMatrix")
    o_delete(f, "flyTable/normalizedSimilarityMatrix")
    o_delete(f, "flyTable/meanSimilarityMatrix")
    
    o_delete(f, "zfishTable/rawSimilarityMatrix")
    o_delete(f, "zfishTable/normalizedSimilarityMatrix")
    o_delete(f, "zfishTable/meanSimilarityMatrix")
    
    o_delete(f, "zfishTable/semantic/rawSimilarityMatrix")
    o_delete(f, "zfishTable/semantic/normalizedSimilarityMatrix")
    o_delete(f, "zfishTable/semantic/meanSimilarityMatrix")
    
    o_delete(f, "zfishTable/small2big/normalizedSimilarityMatrix")
    o_delete(f, "zfishTable/small2big/meanSimilarityMatrix")
    
    o_delete(f, "zfishTable/semantic/small2big/normalizedSimilarityMatrix")
    o_delete(f, "zfishTable/semantic/small2big/meanSimilarityMatrix")
end


remove the dataset first, otherwise we can not write!


0

In [6]:
using Pkg
using HDF5
println("read score tables...")
df = CSV.read(joinpath(Pkg.devdir(), "RealNeuralNetworks/asset/zfish_score_table.csv"))

zfish_ria = RangeIndexingArray{Float32}(df)

df = CSV.read(joinpath(Pkg.devdir(), "RealNeuralNetworks/asset/smat_fcwb.csv"))
fly_ria = RangeIndexingArray{Float32}(df)

k = 10

println("compute similarity score using fly table...")
flyTableSimilarityMatrix, flyTableNormalizedSimilarityMatrix, flyTableMeanSimilarityMatrix  = 
            nblast_allbyall(neuronList; semantic=false, k=k, ria=fly_ria);
h5open("$(DATASET_DIR)/evaluate.h5", "r+") do f
    f["flyTable/rawSimilarityMatrix"] = flyTableSimilarityMatrix
    f["flyTable/normalizedSimilarityMatrix"] = flyTableNormalizedSimilarityMatrix
    f["flyTable/meanSimilarityMatrix"] = flyTableMeanSimilarityMatrix
end

println("compute similarity score using zfish table...")
zfishTableSimilarityMatrix, zfishTableNormalizedSimilarityMatrix, zfishTableMeanSimilarityMatrix  = 
            nblast_allbyall(neuronList; semantic=false, k=k, ria=zfish_ria);
h5open("$(DATASET_DIR)/evaluate.h5", "r+") do f
    f["zfishTable/rawSimilarityMatrix"] = zfishTableSimilarityMatrix
    f["zfishTable/normalizedSimilarityMatrix"] = zfishTableNormalizedSimilarityMatrix
    f["zfishTable/meanSimilarityMatrix"] = zfishTableMeanSimilarityMatrix
end

println("compute similarity score using zfish table with semantic matching...")
zfishTableSemanticSimilarityMatrix, zfishTableSemanticNormalizedSimilarityMatrix, zfishTableSemanticMeanSimilarityMatrix  = 
            nblast_allbyall(neuronList; semantic=true, k=k, ria=zfish_ria);
h5open("$(DATASET_DIR)/evaluate.h5", "r+") do f
    f["zfishTable/semantic/rawSimilarityMatrix"] = zfishTableSemanticSimilarityMatrix
    f["zfishTable/semantic/normalizedSimilarityMatrix"] = zfishTableSemanticNormalizedSimilarityMatrix
    f["zfishTable/semantic/meanSimilarityMatrix"] = zfishTableSemanticMeanSimilarityMatrix
end

println("compute similarity score using zfish table with small-to-big...")
small2bigSimilarityMatrix = NBLASTs.nblast_allbyall_small2big(neuronList; k=k, ria=zfish_ria);
h5open("$(DATASET_DIR)/evaluate.h5", "r+") do f
    f["zfishTable/small2big/similarityMatrix"] = small2bigSimilarityMatrix
end

println("compute similarity score using zfish table with semantic small-to-big...")
semanticSmall2bigSimilarityMatrix = NBLASTs.nblast_allbyall_small2big(neuronList; 
    k=k, ria=zfish_ria, semantic=true);
h5open("$(DATASET_DIR)/evaluate.h5", "r+") do f
    f["zfishTable/semantic/small2big/similarityMatrix"] = semanticSmall2bigSimilarityMatrix
end

println("finished computing similarity matrix!")

read score tables...


┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = RangeIndexingArray{Float32,N} where N(::DataFrames.DataFrame) at RangeIndexingArrays.jl:61
└ @ RealNeuralNetworks.Utils.RangeIndexingArrays /usr/people/jingpeng/.julia/dev/RealNeuralNetworks/src/Utils/RangeIndexingArrays.jl:61


compute similarity score using fly table...
compute similarity score using zfish table...
compute similarity score using zfish table with semantic matching...


┌ Warning: the number of node 0 is less than than the number of neighborhood points k=10, return zero vector
└ @ RealNeuralNetworks.NBLASTs /usr/people/jingpeng/.julia/dev/RealNeuralNetworks/src/NBLASTs.jl:58
┌ Warning: the number of node 0 is less than than the number of neighborhood points k=10, return zero vector
└ @ RealNeuralNetworks.NBLASTs /usr/people/jingpeng/.julia/dev/RealNeuralNetworks/src/NBLASTs.jl:58
┌ Warning: the number of node 0 is less than than the number of neighborhood points k=10, return zero vector
└ @ RealNeuralNetworks.NBLASTs /usr/people/jingpeng/.julia/dev/RealNeuralNetworks/src/NBLASTs.jl:58
┌ Warning: the number of node 0 is less than than the number of neighborhood points k=10, return zero vector
└ @ RealNeuralNetworks.NBLASTs /usr/people/jingpeng/.julia/dev/RealNeuralNetworks/src/NBLASTs.jl:58
┌ Warning: the number of node 0 is less than than the number of neighborhood points k=10, return zero vector
└ @ RealNeuralNetworks.NBLASTs /usr/people/jingpeng/.ju

computing similarity matrix...100%|█████████████████████| Time: 0:00:11


compute similarity score using zfish table with small-to-big...
compute similarity score using zfish table with semantic small-to-big...


┌ Warning: the number of node 0 is less than than the number of neighborhood points k=10, return zero vector
└ @ RealNeuralNetworks.NBLASTs /usr/people/jingpeng/.julia/dev/RealNeuralNetworks/src/NBLASTs.jl:58
┌ Warning: the number of node 0 is less than than the number of neighborhood points k=10, return zero vector
└ @ RealNeuralNetworks.NBLASTs /usr/people/jingpeng/.julia/dev/RealNeuralNetworks/src/NBLASTs.jl:58
┌ Warning: the number of node 0 is less than than the number of neighborhood points k=10, return zero vector
└ @ RealNeuralNetworks.NBLASTs /usr/people/jingpeng/.julia/dev/RealNeuralNetworks/src/NBLASTs.jl:58
┌ Warning: the number of node 0 is less than than the number of neighborhood points k=10, return zero vector
└ @ RealNeuralNetworks.NBLASTs /usr/people/jingpeng/.julia/dev/RealNeuralNetworks/src/NBLASTs.jl:58
┌ Warning: the number of node 0 is less than than the number of neighborhood points k=10, return zero vector
└ @ RealNeuralNetworks.NBLASTs /usr/people/jingpeng/.ju

finished computing similarity matrix!


# Evaluate clustering

In [7]:
# perform clustering in R


# Investigate a neuron pair

In [8]:
# neuronId1 = 76631
# neuronId2 = 82169
# neuronId1 = 77710
# neuronId2 = 77302

neuronId1 = 77634
neuronId2 = 79066

neuron1 = Neurons.load("$(DATASET_DIR)/bin/$(neuronId1).bin");
neuron2 = Neurons.load("$(DATASET_DIR)/bin/$(neuronId2).bin");
n1 = Neurons.get_num_nodes(neuron1)
println("neuron $neuronId1 has $(n1) nodes.")
neuron1 = Neurons.resample(neuron1, Float32(1000))
n1 = Neurons.get_num_nodes(neuron1)
println("neuron $neuronId1 has $(n1) nodes.")

n2 = Neurons.get_num_nodes(neuron2)
println("neuron $neuronId2 has $(n2) nodes.")
neuron2 = Neurons.resample(neuron2, Float32(1000))
n2 = Neurons.get_num_nodes(neuron2)
println("neuron $neuronId2 has $(n2) nodes.")

neuron 77634 has 768 nodes.
neuron 77634 has 690 nodes.
neuron 79066 has 694 nodes.
neuron 79066 has 619 nodes.


In [9]:
using Plotly
import Pkg
include(joinpath(Pkg.devdir(), "RealNeuralNetworks/src/Utils/PlotRecipes.jl"))

neuronId = 77886
neuron = Neurons.load("$(DATASET_DIR)/bin/$(neuronId).bin")
my_plot = PlotRecipes.plot(neuron; nodeStep=1)

remote_plot = Plotly.post(my_plot)
remote_plot.url.uri

ArgumentError: ArgumentError: Package Plotly not found in current path:
- Run `import Pkg; Pkg.add("Plotly")` to install the Plotly package.


In [10]:
NBLASTs.nblast(neuron1, neuron2)

-9463.488f0

In [11]:
NBLASTs.nblast(neuron2, neuron1)

-10528.503f0

In [12]:
neuronId3 = 77705

function compute_nblast_score(neuronId1, neuronId2)
    neuron1 = Neurons.load("$(DATASET_DIR)/atlas/bin/$(neuronId1).bin")
    neuron2 = Neurons.load("$(DATASET_DIR)/atlas/bin/$(neuronId2).bin")
    neuron1 = Neurons.resample(neuron1, Float32(1000))
    neuron1 = Neurons.resample(neuron1, Float32(1000))
    score12 = NBLASTs.nblast(neuron1, neuron2)
    score21 = NBLASTs.nblast(neuron2, neuron1)
    
    
    if Neurons.get_num_nodes(neuron1) > Neurons.get_num_nodes(neuron2)
        scoresb = score12
    else
        scoresb = score21
    end
    
    println("$(neuronId1) and $neuronId2 score is $(score12)")
    println("$(neuronId2) and $neuronId1 score is $(score21)")
    println("small to big score is $(scoresb)")
    return score12, score21, scoresb
end

compute_nblast_score(77640, 77710);

77640 and 77710 score is -7163.628
77710 and 77640 score is -9363.354
small to big score is -7163.628


In [13]:
compute_nblast_score(77302, 77710);

77302 and 77710 score is -7175.5444
77710 and 77302 score is -3462.4004
small to big score is -3462.4004
